# The War Room

Simulation & training notebook for the **War on Art** auction strategy engine.

Use this notebook to simulate battles before connecting to the real Catawiki MCP.

In [ ]:
# SIMULATION CELL
from strategy.inputs import AuctionParams
from strategy.engine import WarOnArtEngine
from strategy.learner import ReinforcementLearner

# 1. Configure Codruta's Intent
params = AuctionParams(
    lot_id="12345",
    lot_url="https://catawiki...",
    max_budget=500,     # She wants to spend max €500
    greediness=85       # She REALLY wants this (High Greed)
)

print(f"\U0001f4b0 Visible Budget: \u20ac{params.max_budget}")
print(f"\U0001f6e1\ufe0f True Ceiling (Ozymandias Offset): \u20ac{params.true_ceiling}")

# 2. Initialize Engine
learner = ReinforcementLearner()
engine = WarOnArtEngine(params, learner)

# 3. Simulate a Scenario: "The Dead Zone"
# Opponent bid \u20ac480, 10 seconds left, We are NOT leading
current_bid = 480
time_left = 10
is_leading = False

action, amount, reason = engine.evaluate_state(current_bid, time_left, is_leading)

print(f"\n\u26a1 TACTICAL DECISION: {action}")
print(f"\U0001f4b6 Bid Amount: \u20ac{amount}")
print(f"\U0001f4dd Reasoning: {reason}")